## Exercício Desafio

- Digamos que seu chefe pediu para você um relatório da análise dos salários da unidade de San Francisco da empresa. O objetivo dele é entender:

1. Qual foi a evolução do salário médio ao longo dos anos (TotalPay e TotalPayBenefits)
2. Quantos funcionários tivemos ao longo dos anos
3. Qual foi a evolução do total gasto com salário ao longo dos anos (TotalPayBenefits)

- Base de Dados a ser usada: salarios.sqlite

### Importação da Base de Dados (sqlite3)

In [43]:
import pandas as pd
import sqlite3

conexao = sqlite3.connect('salarios.sqlite')

tabela_salarios = pd.read_sql('SELECT * FROM Salaries', conexao)
# tabela_salarios = tabela_salarios.drop(columns=["Id"])
print("Conexão efetuada com sucesso!")

conexao.close()

display(tabela_salarios)

Conexão efetuada com sucesso!


,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0,400184.25,,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.71,198306.9,,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737,182234.59,,326373.19,326373.19,2011,,San Francisco,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148649,148650,Roy I Tillery,Custodian,0,0,0,0,0.00,0.00,2014,,San Francisco,PT
148650,148651,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,
148651,148652,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,
148652,148653,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,


### Importação da Base de Dados (pyodbc)

In [ ]:
import pyodbc

# print(pyodbc.drivers())

dados_conexao = (
    "Driver={SQLite3 ODBC Driver};"
    "Server=localhost;"
    "Database=salarios.sqlite;"
)

conexao = pyodbc.connect(dados_conexao)
print("Conexão efetuada com sucesso!")

cursor = conexao.cursor()

cursor.execute('SELECT * FROM Salaries')

valores = cursor.fetchall() # retornado uma lista de tuplas
descricao = cursor.description

cursor.close()
conexao.close()

In [ ]:
colunas = [tupla[0] for tupla in descricao]
print(colunas)

In [ ]:
tabela_salarios2 = pd.DataFrame.from_records(valores, columns=colunas)
tabela_salarios2 = tabela_salarios2.drop(columns=["Id"])
display(tabela_salarios2)

### Análise de Dados

In [47]:
# garantindo que estamos só com San Francisco

tabela_salarios_sanfrancisco = tabela_salarios.loc[tabela_salarios["Agency"]=="San Francisco", :]

# removendo registros Nulos e com 'Not provided' nas colunas TotalPay e JobTItle
tabela_salarios_sanfrancisco_no_null_values = tabela_salarios_sanfrancisco.loc[
    (tabela_salarios_sanfrancisco["JobTitle"] != "Not provided") &
    (tabela_salarios_sanfrancisco["BasePay"] != 0) &
    (tabela_salarios_sanfrancisco["TotalPay"] != 0) &
    (tabela_salarios_sanfrancisco["TotalPayBenefits"] != 0), :
]

display(tabela_salarios_sanfrancisco_no_null_values)

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0,400184.25,,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.71,198306.9,,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737,182234.59,,326373.19,326373.19,2011,,San Francisco,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148613,148614,Nanette M Lloyd,Special Nurse,19.97,0,0,4.3,19.97,24.27,2014,,San Francisco,PT
148616,148617,Afakasi O Afakasi,Public Service Trainee,21.48,0,0.86,0.22,22.34,22.56,2014,,San Francisco,PT
148617,148618,Charles F Williams Iii,Pool Lifeguard,17.26,0,0,4.04,17.26,21.30,2014,,San Francisco,PT
148619,148620,Ian V Cameron,IS Program Analyst-Assistant,6.04,0,10.05,2.3,16.09,18.39,2014,,San Francisco,PT


In [48]:
print(tabela_salarios_sanfrancisco_no_null_values.info())

print(tabela_salarios_sanfrancisco_no_null_values.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 147064 entries, 0 to 148620
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                147064 non-null  int64  
 1   EmployeeName      147064 non-null  object 
 2   JobTitle          147064 non-null  object 
 3   BasePay           147064 non-null  object 
 4   OvertimePay       147064 non-null  object 
 5   OtherPay          147064 non-null  object 
 6   Benefits          147064 non-null  object 
 7   TotalPay          147064 non-null  float64
 8   TotalPayBenefits  147064 non-null  float64
 9   Year              147064 non-null  int64  
 10  Notes             147064 non-null  object 
 11  Agency            147064 non-null  object 
 12  Status            147064 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 15.7+ MB
None
Id                  0
EmployeeName        0
JobTitle            0
BasePay             0
OvertimePay       

##### 1. Qual foi a evolução do salário médio ao longo dos anos

In [49]:
# agrupando a média do total de salários e total de salários com beneficios com base nos anos
tabela_salarios_medios = tabela_salarios_sanfrancisco_no_null_values.groupby("Year").mean(numeric_only=True)
display(tabela_salarios_medios[["TotalPay", "TotalPayBenefits"]])

,TotalPay,TotalPayBenefits
Year,,
2011,72595.460609,72595.460609
2012,74905.619412,101643.045310
2013,78210.405992,102162.505491
2014,76279.728792,101329.827838


##### 2. Quantos funcionários tivemos ao longo dos anos

In [52]:
tabela_qtde_func = tabela_salarios_sanfrancisco.groupby("Year").count()
tabela_qtde_func = tabela_qtde_func[["Id"]]
tabela_qtde_func = tabela_qtde_func.rename(columns={"Id": "Qtd. Funcionários"})
display(tabela_qtde_func)

,Qtd. Funcionários
Year,
2011,36159
2012,36766
2013,37606
2014,38123


##### 3. Qual foi a evolução do total gasto com salário ao longo dos anos

In [57]:
def formatar_valor(valor):
    return f'R$ {valor:,.2f}'

tabela_total_gasto_salario = tabela_salarios_sanfrancisco.groupby("Year").sum(numeric_only=True)
tabela_total_gasto_salario = tabela_total_gasto_salario[["TotalPay", "TotalPayBenefits"]]
tabela_total_gasto_salario["TotalPay"] = tabela_total_gasto_salario["TotalPay"].apply(formatar_valor)
tabela_total_gasto_salario["TotalPayBenefits"] = tabela_total_gasto_salario["TotalPayBenefits"].apply(formatar_valor)
display(tabela_total_gasto_salario)

,TotalPay,TotalPayBenefits
Year,,
2011,"R$ 2,594,195,051.88","R$ 2,594,195,051.88"
2012,"R$ 2,724,848,200.44","R$ 3,696,940,025.96"
2013,"R$ 2,918,655,930.80","R$ 3,814,772,184.37"
2014,"R$ 2,876,910,951.26","R$ 3,821,865,780.60"
